# SET UP

In [1]:

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [19]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    
    # name: Optional[str] = Field(
    #     default=None, description="The name of the person"
    # )
    name: str = Field(description="Include only a person's first name. Do not include if they have a middle name")
    lastname: Optional[str] = Field(
        default=None, description="The lastname of the person if known"
    )
    country: Optional[str] = Field(
        default=None, description="The country of the person if known"
    )

In [ ]:
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) You can introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

In [5]:
chain = prompt | llm.with_structured_output(schema=Person)

c:\Users\FranciscoMartinezPoy\Desktop\proyectos\GenAI\venv_ia\Lib\site-packages\langchain_openai\chat_models\base.py:1360: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [6]:
import os 
os.getcwd()

'c:\\Users\\FranciscoMartinezPoy\\Desktop\\proyectos\\GenAI'

In [ ]:
import pdfplumber

# Ruta al archivo PDF
ruta_pdf = 'docs/CV_Francisco_M_Martinez_Poyatos_2024.pdf'

# Abrir el archivo PDF
with pdfplumber.open(ruta_pdf) as pdf:
    # Iterar sobre cada página
    for num_pagina, pagina in enumerate(pdf.pages):
        # Extraer texto de la página
        texto = pagina.extract_text()
        #print(f"Texto de la página {num_pagina + 1}:\n{texto}\n")

In [20]:
chain.invoke({"text":texto})

Person(name='Francisco Mª', lastname='Martínez Poyatos', country=None)